# Examining the Effects of DP on Exponential Smoothing Forecast Accuracy

***

In [1]:
# general modules
import pandas as pd
import numpy as np
import sktime

# import exponential smoothing forecasting model
from sktime.forecasting.exp_smoothing import ExponentialSmoothing

# functions for transformation+forecasting pipeline
from sktime.forecasting.compose import TransformedTargetForecaster

# time series transformations
from sktime.transformations.series.detrend import ConditionalDeseasonalizer

##### the `helper_functions.py` file contains many custom functions we wrote to aid in our analysis
##### `full_coding_analysis` combines all of the following - train-test split data,
##### data protection, train models, compare accuracies, return accuracy results
from helper_functions import *

# suppress warnings from exponential smoothing model not converging
import warnings
warnings.filterwarnings('ignore')

In [2]:
# import weekly finance time series
Y = np.genfromtxt("../../Data/Train/Clean/weekly_finance_clean.csv", delimiter = ',', skip_header = 1)
Y = pd.DataFrame(Y)

In [3]:
# detrender = Detrender()
# detrended_series = [detrender.fit_transform(series) for _ , series in Y.iterrows()]
# detrended_series = [i+np.abs(np.min(i))+1.0 for i in detrended_series]
# Y = pd.concat(detrended_series, axis=1).T

***

## SES

In [4]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        ("forecast", ExponentialSmoothing(use_boxcox=False)),
    ]
)

In [5]:
results_dict_ses = {}
fcasts_ses = {}
fcasts_protected_ses = {}
tests = {}
epsilons = [1, 10, 20]
horizons = [1, 20]

In [6]:
for e in epsilons:
    for h in horizons:
        idx = "h="+str(h)+", epsilon = "+str(e)
        results_dict_ses[idx], tests[idx], fcasts_ses[idx], fcasts_protected_ses[idx] = full_coding_analysis(time_series_data=Y, 
                                                                                                             forecasting_model=forecaster, 
                                                                                                             forecast_horizon=h,
                                                                                                             epsilon=e)

In [7]:
results_dict_ses

{'h=1, epsilon = 1': {'Mean Accuracies': array([0.0218, 0.0218]),
  'Protected Mean Accuracies:': array([1.2811, 1.2811]),
  '% Change Mean accuracy:': array([-57.8465, -57.8465]),
  '% Change Median accuracy:': array([-120.9714, -120.9714]),
  '% Forecasted Points adjusted downward:': 0.0427,
  '% Forecasted Points adjusted upward:': 0.9573,
  '% Series with improved accuracy:': array([0.0122, 0.0122]),
  '% Series with reduced accuracy:': array([0.9878, 0.9878]),
  'Original Mean Absolute Error Upward Adjusted:': 0.0225,
  'Original Mean Absolute Error Downward Adjusted:': 0.0051,
  'Protected Mean Absolute Error Upward Adjusted:': 1.3354,
  'Protected Mean Absolute Error Downward Adjusted:': 0.0643},
 'h=20, epsilon = 1': {'Mean Accuracies': array([0.0445, 0.0543]),
  'Protected Mean Accuracies:': array([1.2224, 1.2233]),
  '% Change Mean accuracy:': array([-26.4805, -21.5269]),
  '% Change Median accuracy:': array([-33.8758, -28.1642]),
  '% Forecasted Points adjusted downward:': 0

***
***

In [8]:
original_forecasts = fcasts_ses['h=20, epsilon = 20']
protected_forecasts = fcasts_protected_ses['h=20, epsilon = 20']
test = tests['h=20, epsilon = 20']

In [9]:
adjusted_up = original_forecasts < protected_forecasts
adjusted_up = pd.concat([row for i, row in adjusted_up.iterrows()])
adjusted_down = original_forecasts > protected_forecasts
adjusted_down = pd.concat([row for i, row in adjusted_down.iterrows()])

In [10]:
absolute_error_original = np.absolute(test - original_forecasts)
absolute_error_protected = np.absolute(test - protected_forecasts)

In [11]:
improved = absolute_error_original > absolute_error_protected
improved = pd.concat([row for i, row in improved.iterrows()])
worsened = absolute_error_original < absolute_error_protected
worsened = pd.concat([row for i, row in worsened.iterrows()])

In [12]:
np.mean(adjusted_down[improved])

0.46419951729686243

In [13]:
np.mean(adjusted_up[improved])

0.5358004827031375

***
***

## DES

In [14]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        ("forecast", ExponentialSmoothing(trend="additive", use_boxcox=False)),
    ]
)

In [15]:
results_dict_des = {}
fcasts_des = {}
fcasts_protected_des = {}
tests = {}
epsilons = [1, 10, 20]
horizons = [1, 20]

In [16]:
for e in epsilons:
    for h in horizons:
        idx = "h="+str(h)+", epsilon = "+str(e)
        results_dict_des[idx], tests[idx], fcasts_des[idx], fcasts_protected_des[idx] = full_coding_analysis(time_series_data=Y, 
                                                                                                             forecasting_model=forecaster, 
                                                                                                             forecast_horizon=h,
                                                                                                             epsilon=e)

In [17]:
results_dict_des

{'h=1, epsilon = 1': {'Mean Accuracies': array([0.021, 0.021]),
  'Protected Mean Accuracies:': array([1.2815, 1.2815]),
  '% Change Mean accuracy:': array([-60.1278, -60.1278]),
  '% Change Median accuracy:': array([-114.7976, -114.7976]),
  '% Forecasted Points adjusted downward:': 0.0244,
  '% Forecasted Points adjusted upward:': 0.9756,
  '% Series with improved accuracy:': array([0.0061, 0.0061]),
  '% Series with reduced accuracy:': array([0.9939, 0.9939]),
  'Original Mean Absolute Error Upward Adjusted:': 0.0214,
  'Original Mean Absolute Error Downward Adjusted:': 0.004,
  'Protected Mean Absolute Error Upward Adjusted:': 1.3121,
  'Protected Mean Absolute Error Downward Adjusted:': 0.0594},
 'h=20, epsilon = 1': {'Mean Accuracies': array([0.0428, 0.0524]),
  'Protected Mean Accuracies:': array([1.223 , 1.2241]),
  '% Change Mean accuracy:': array([-27.5504, -22.3708]),
  '% Change Median accuracy:': array([-37.5498, -30.1769]),
  '% Forecasted Points adjusted downward:': 0.03

***
***

## TES

In [18]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        ("forecast", ExponentialSmoothing(trend="additive",
                                          seasonal="additive",
                                          sp=52,
                                          damped_trend=False, 
                                          use_boxcox=False)),
    ]
)

In [19]:
results_dict_tes = {}
fcasts_tes = {}
fcasts_protected_tes = {}
tests = {}
epsilons = [1, 10, 20]
horizons = [1, 20]

In [20]:
for e in epsilons:
    for h in horizons:
        idx = "h="+str(h)+", epsilon = "+str(e)
        results_dict_tes[idx], tests[idx], fcasts_tes[idx], fcasts_protected_tes[idx] = full_coding_analysis(time_series_data=Y, 
                                                                                                             forecasting_model=forecaster, 
                                                                                                             forecast_horizon=h,
                                                                                                             epsilon=e)

In [21]:
results_dict_tes

{'h=1, epsilon = 1': {'Mean Accuracies': array([0.0244, 0.0244]),
  'Protected Mean Accuracies:': array([1.2961, 1.2961]),
  '% Change Mean accuracy:': array([-52.0754, -52.0754]),
  '% Change Median accuracy:': array([-108.2696, -108.2696]),
  '% Forecasted Points adjusted downward:': 0.0183,
  '% Forecasted Points adjusted upward:': 0.9817,
  '% Series with improved accuracy:': array([0.0122, 0.0122]),
  '% Series with reduced accuracy:': array([0.9878, 0.9878]),
  'Original Mean Absolute Error Upward Adjusted:': 0.0248,
  'Original Mean Absolute Error Downward Adjusted:': 0.0056,
  'Protected Mean Absolute Error Upward Adjusted:': 1.3187,
  'Protected Mean Absolute Error Downward Adjusted:': 0.082},
 'h=20, epsilon = 1': {'Mean Accuracies': array([0.0475, 0.0574]),
  'Protected Mean Accuracies:': array([1.2392, 1.2468]),
  '% Change Mean accuracy:': array([-25.1041, -20.7061]),
  '% Change Median accuracy:': array([-34.7004, -27.9298]),
  '% Forecasted Points adjusted downward:': 0.